# Electronic Structure Calculations using Generalized Ising Hamiltonians

Your final task is to calculate the ground state energy of a hydrogen molecule using a Generalized Ising Hamiltonian:

$$ H = E_0 + \sum_i h_i\sigma_i + \sum_{ij} J_{ij}\sigma_i\sigma_j + \sum_{ijk} K_{ijk}\sigma_i\sigma_j\sigma_k + \sum_{ijkl} L_{ijkl}\sigma_i\sigma_j\sigma_k\sigma_l + \cdots $$

where the Hamiltonian parameters ($E_0, h_i, J_{ij}, K_{ijk}, L_{ijkl}$) will be provided to you by the `read_generalized_ising_hamiltonian` function below.

These Ising Hamiltonians were produced using the *Iterative Qubit Coupled Cluster* method (https://arxiv.org/abs/1906.11192).

The $H_2$ Ising Hamiltonian only needs 4 spins to fully describe the ground state, hence our Hamiltonian only needs a 4-point interaction at most.

In [1]:
import numpy as np

from abstract_ising import AbstractIsing
from ising_animator import IsingAnimator

%matplotlib inline

In [3]:
def read_generalized_ising_hamiltonian(path):
    with open(path, "r") as f:
        f.readline()  # discard first line
        compressed_hamiltonian = [
            tuple(line.strip().split())
            for line in f.readlines()
        ]
    
    num_sites = len(compressed_hamiltonian[0][0])
    hamiltonian_terms = [np.zeros((num_sites,)*i) for i in range(num_sites+1)]

    for sites, val in compressed_hamiltonian:
        num_zs = 0
        site_nums = []
        for i, x in enumerate(sites):
            if x == 'z':
                site_nums.append(i)
                num_zs += 1

        hamiltonian_terms[num_zs][tuple(site_nums)] = float(val)

    return hamiltonian_terms

In [4]:
E0, h, J, K, L = read_generalized_ising_hamiltonian("./hamiltonians/Ising-H2-STO-3G-bk-samespin-R=0.65.inp")

First, you must write a class that describes the Generalized Ising Model given the arrays containing the
 Hamiltonian parameters. You will need to write a function which computes energy of the stored spin configuration
 (see the previous tasks for inspiration), as well as a function which computes the change in energy resulting from
 a single-spin-flip (you could of course use the naive approach and simply compute the energy of two spin configurations
  and subtract one from the other).

In [68]:
spins = 2 * (np.random.rand(4) < 0.5) - 1

res = 0
for i, s0 in enumerate(spins):
    for j, s1 in enumerate(np.roll(spins, 1)):
        for k, s2 in enumerate(np.roll(spins, 2)):
                res += s0 * s1 * s2*  K[i][j][k]
print(res)
np.einsum(spins, [0],  np.roll(spins, 1), [1],  np.roll(spins, 2), [2],  K, [0,1,2])

0.6214728053893763


0.6214728053893763

In [107]:
spins = 2 * (np.random.rand(4) < 0.5) - 1
terms = (E0, h, J, K, L)

energy = []
for dim, term in enumerate(terms):
    prog = []
    for d in range(dim):
        prog.extend([np.roll(spins, d), [d]])
    prog.extend([term, list(range(dim))])
    energy.append(np.einsum(*prog))

energy1 = []
energy1.append(float(E0))
energy1.append(np.einsum('i, i', h, spins))
energy1.append(np.einsum('i,j,ij', spins, np.roll(spins, 1), J))
energy1.append(np.einsum('i,j,k, ijk', spins, np.roll(spins, 1), np.roll(spins, 2), K))
energy1.append(np.einsum('i,j,k,l, ijkl', spins, np.roll(spins, 1), np.roll(spins, 2), np.roll(spins, 3), L))

print(energy)
print(energy1)
print(np.testing.assert_array_equal(energy, energy1))

[0.03775117547636375, -0.13421845044578246, -0.32839586407884513, 0.07570471329069535, 0.1699209802656651]
[0.03775117547636375, -0.13421845044578246, -0.32839586407884513, 0.07570471329069535, 0.1699209802656651]
None


In [116]:
J[0][-3]

0.0

In [117]:
spins = 2 * (np.random.rand(4) < 0.5) - 1
terms = (E0, h, J, K, L)
def energy():
    energy = 0
    for dim, term in enumerate(terms):
        prog = []
        for d in range(dim):
            prog.extend([np.roll(spins, d), [d]])
        prog.extend([term, list(range(dim))])
        energy += np.einsum(*prog)
    return energy

def energy_diff(*coords):
    #naieve approach for now
    e0 = energy()
    spins[coords] *= -1
    e1 = energy()
    diff = e1 - e0
    spins[coords] *= -1
    return diff

def energy_diff2(ind):
    hdiff = terms[1][ind]
    jdiff = (terms[2][ind-1][ind])
    jdiff += (terms[2][ind][ind+ 1])
    kdiff = (terms[3][ind-2][ind-1][ind])
    kdiff += (terms[3][ind+2][ind+1][ind])
    ldiff = (terms[4][ind-3][ind-2][ind-1][ind])
    ldiff += (terms[4][ind+3][ind+2][ind+1][ind])
    energy = 2 * (hdiff + jdiff + kdiff + ldiff)
    return energy
for i in range(4):
    print('\ni = {}'.format(i))
    print(energy_diff(i))
    print(energy_diff2(i))


i = 0
-0.2773311912071159
0.0

i = 1
-1.1566998702711828


IndexError: index 4 is out of bounds for axis 0 with size 4

In [166]:
# Write a class to describe the generalized Ising Model given its Hamiltonian parameters

class GeneralizedIsingModel(AbstractIsing):
    def __init__(self, terms):
        self.terms = terms
        for t0, t1 in zip(terms, terms[1:]):
            if len(t1.shape) < len(t0.shape):
                raise ValueError('The terms have not been provided in the right order. Please provide them in order of '
                                 'increasing dimension')

        self.num_spins = terms[1].shape[0]
        self.spins = 2 * (np.random.rand(self.num_spins) < 0.5) - 1
    
    def energy(self):
        energy = 0
        for dim, term in enumerate(self.terms):
            prog = []
            for d in range(dim):
                prog.extend([np.roll(self.spins, d), [d]])
            prog.extend([term, list(range(dim))])
            energy += np.einsum(*prog)
        return energy
    
    def energy_diff(self, *coords):
        #naieve approach for now
        e0 = self.energy()
        self.spins[coords] *= -1
        e1 = self.energy()
        diff = e1 - e0
        self.spins[coords] *= -1
        return diff
    
    def rand_site(self):
        return [np.random.randint(self.num_spins)]

initial: 0.048813664877280324
spins: [-1 -1 -1  1]



KeyboardInterrupt: 

Next you'll run a Monte Carlo simulation for this model at some finite temperature for 1000 steps, printing out the energy of the state every so often

In [167]:
terms = [E0, h, J, K, L]
hydrogen = GeneralizedIsingModel(terms)
for t in range(1000):
    # take a look at the abstract_ising.py file to see how mc_step works
    E = hydrogen.mc_step(T=1)

    if t % 50 == 0:
        print(E)

-0.33319337985945596
-0.06220997118172915
-1.147398372849702
0.04881366487728034
-0.17923744549190346
-1.224913679704077
-0.333193379859456
-1.224913679704077
-0.8429066349673405
-0.7653913281129656
-1.224913679704077
-1.224913679704077
-0.4442170159184655
0.4351516631456014
-0.8429066349673406
-0.4442170159184656
-0.06220997118172916
0.7124828543527175
0.2027695992448328
-0.06220997118172922


Now, apply (one of) the annealing procedure(s) you came up with in the previous task to this problem to find a ground state of the system:

In [6]:
# your annealing code here

Finally, iterate over the entire spin configuration space (this is tractable since we only have 4 spins) to find the exact ground state energy. Compare this energy to the one you found above using your annealer.

In [7]:
# exact ground state calculation

Now, clean up your code a little, and write a for-loop that iterates over all the different values of the Hydrogen seperation distance $R$ available in the `hamiltonians` directory. 

For each $R$ you must:
- Read in the associated Ising Hamiltonian
- Perform an annealed Monte Carlo simulation to find a candidate ground state energy
- Compute the exact ground state energy
- Compare the two results